Assess the influence of left truncation on Kaplan Meier

In [ ]:
from lifelines.plotting import plot_lifetimes
from lifelines import KaplanMeierFitter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, inspect, sys

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import epri_mc_lib_3 as mc
from importlib import reload
reload(mc)

In [ ]:
data = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/Survival_df.csv'),
                  index_col=0)
data_GAN = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/CopulaGAN_simulated_data_survival_2.csv'),
                  index_col=None)

data_x = data.iloc[:, 2:]
data_y = data.iloc[:, 0:2]

data_GAN_x = data_GAN.iloc[:, 2:]
data_GAN_y = data_GAN.iloc[:, 0:2]

In [ ]:
merge = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/MERGE_on_sample.csv'), 
                   index_col=0)
merge.loc[merge.F_Time.isna(), 'unfailed_cycle'] = merge.NDE_cycle
merge['failure'] = merge.failure_cycle
merge.failure = merge.failure.fillna(merge.unfailed_cycle)
merge.dropna(subset=['failure'], inplace=True)
merge.drop(['failure_cycle', 'unfailed_cycle', 'F_Fraction', 'F_Time'], axis=1, inplace=True)

In [ ]:
_, ax = plt.subplots(figsize=(5,7))
plot_lifetimes(
    merge['failure'] - merge['NDE_cycle'],
    event_observed=merge['Observed'],
    entry=merge['NDE_cycle'],
    event_observed_color="#8f1402",
    event_censored_color="#029386",
    left_truncated=True,
)
plt.ylabel('Sample ID')
plt.xlabel('Total cycle run')

In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(merge['failure'], event_observed = merge['Observed'], entry=merge['NDE_cycle'], label='modeling late entries')
ax = kmf.plot()

kmf.fit(merge['failure'], event_observed = merge['Observed'], label='ignoring late entries')
kmf.plot(ax=ax)

plt.ylabel("est. probability of survival $\hat{S}(t)$")
